test a tidy script that fetches all required datasets to a datastack

In [71]:
### fiddling with GEE and returning workable outputs can be a faff
### to get e.g. band names as a python list, need to call getInfo() afterwards e.g. 
#s1.bandNames().getInfo()
#returns: ['S1_VV_2018-12-01_2019-02-01', 'S1_VH_2018-12-01_2019-02-01']

In [1]:
import ee
from geemap import geemap
ee.Initialize()

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def fetch_sentinel1_v1(aoi, start_date_list):
    """
    fetch a datastack of Sentinel-1 monthly composites.
    
    :param aoi: ee.featurecollection.FeatureCollection, used to indicate AOI extent 
    :param start_date_list: list, strings used to define start of each month, expects 'YYYY-MM-01' format
    :return: ee.image.Image, stack of monthly composite images
    """
    print('fetch_sentinel1(): hello!')
    
    # specify filters to apply to the GEE Sentinel-1 collection
    filters = [ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
           ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
           ee.Filter.equals("instrumentMode", "IW"),
           ee.Filter.geometry(aoi)]
    
    # iteratively fetch each month of Sentinel-1 imagery and generate a median composite for the AOI
    for i, start_date in enumerate(start_date_list):
        print(f'fetch_sentinel1(): processing month {start_date}')
        end_date = ee.Date(start_date).advance(1, 'month')

        # load and filter collection
        s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
             .filterDate(start_date, end_date)   
        s1 = s1.filter(filters)

        # make composite, clip and give sensible name
        s1_median = (s1.select('VV', 'VH')
                      .median()
                      .clip(aoi.geometry())
                      .rename(f'S1_VV_{start_date[0:7]}', 
                              f'S1_VH_{start_date[0:7]}'))

        # append to stack
        if i == 0:
            median_stack = s1_median
        else:
            median_stack = median_stack.addBands(s1_median)
    
    print('fetch_sentinel1(): bye!')    
    return median_stack

In [4]:
def fetch_sentinel1_v2(aoi, date_list):
    """
    fetch a datastack of Sentinel-1 composites.
    
    :: NEW FOR V2 ::
    * compositing period start and end dates need to be explicitly stated in 'date_list' (monthly composites no longer assumed).
    
    :param aoi: ee.featurecollection.FeatureCollection, used to indicate AOI extent 
    :param date_list: list of tuples of strings (i.e. [('a','b'),('c','d')]), used to define start & end of each compositing period, expects 'YYYY-MM-DD' format
    :return: ee.image.Image, stack of composite images
    """
    print('fetch_sentinel1(): hello!')
    
    S1BANDS = ['VV', 'VH']
    
    # specify filters to apply to the GEE Sentinel-1 collection
    filters = [ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
           ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
           ee.Filter.equals("instrumentMode", "IW"),
           ee.Filter.geometry(aoi)]
    
    # iteratively fetch each month of Sentinel-1 imagery and generate a median composite for the AOI
    for i, date_tuple in enumerate(date_list):
        print(f'fetch_sentinel1(): processing period: {date_tuple[0]} to {date_tuple[1]}')
        new_band_names = [f'S1_{x}_{date_tuple[0]}_{date_tuple[1]}' for x in S1BANDS]
        start_date = ee.Date(date_tuple[0])
        end_date = ee.Date(date_tuple[1])
        
        # load and filter collection
        s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
             .filterDate(start_date, end_date)   
        s1 = s1.filter(filters)

        # make composite, clip and give sensible name
        s1_median = (s1.select(S1BANDS)
                      .median()
                      .clip(aoi.geometry())
                      .rename(new_band_names))

        # append to stack
        if i == 0:
            median_stack = s1_median
        else:
            median_stack = median_stack.addBands(s1_median)
    
    print('fetch_sentinel1(): bye!')    
    return median_stack

In [5]:
def fetch_sentinel2_v1(aoi, start_date_list, s2_params):

    """
    fetch a datastack of Sentinel-2 monthly composites, with cloud/shadow masking applied.
    most of the code to do this is derived from here:
    https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
    
    :param aoi: ee.featurecollection.FeatureCollection, used to indicate AOI extent 
    :param start_date_list: list, strings used to define start of each month, expects 'YYYY-MM-01' format
    :param s2_params: dict, contains parameters used for cloud & shadow masking   
    :return: ee.image.Image, stack of monthly composite images with bands: 
             ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
    """ 
    
    print('fetch_sentinel2(): hello!')
    
    def get_s2_sr_cld_col(aoi, start_date, end_date):
        """
        get & join the S2_SR and S2_CLOUD_PROBABILITY collections
        
        uses globals: 
            CLOUD_FILTER: max cloud coverage (%) permitted in a scene
        
        :returns: ee.ImageCollection
        """
        # Import and filter S2 SR.
        s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterBounds(aoi)
            .filterDate(start_date, end_date)
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

        # Import and filter s2cloudless.
        s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
            .filterBounds(aoi)
            .filterDate(start_date, end_date))

        # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
        return ee.ImageCollection(
            ee.Join.saveFirst('s2cloudless').apply(**{
                'primary': s2_sr_col,
                'secondary': s2_cloudless_col,
                'condition': ee.Filter.equals(**{
                    'leftField': 'system:index',
                    'rightField': 'system:index'
                })}))

    
    def add_cld_shdw_mask(img):
        """ 
        generate a cloud and shadow mask band 
        uses globals: 
            BUFFER: distance (m) used to buffer cloud edges
        :returns: img with added cloud mask, shadow mask, and cloud-shadow mask 
        """
        
        # Add cloud component bands.
        img_cloud = add_cloud_bands(img)

        # Add cloud shadow component bands.
        img_cloud_shadow = add_shadow_bands(img_cloud)

        # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
        is_cld_shdw = (img_cloud_shadow.select('clouds')
                       .add(img_cloud_shadow.select('shadows')).gt(0)
                      )

        # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
        # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
        # mdj TODO: confirmation that BUFFER is in [m]
        #           focal_max() default units = pixels (and pix res is 10m)
        #           so if BUFFER = 100
        #           100 * 0.1 = 10 pixels
        #           10 pix * 10 [pix res] = 100m
        is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
            .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
            .rename('cloudshadowmask'))

        # Add the final cloud-shadow mask to the image.
        return img_cloud_shadow.addBands(is_cld_shdw)    

       
    def add_cloud_bands(img):
        """
        identify cloudy pixels using s2cloudless product probabilty band
        
        uses globals:
            CLD_PRB_THRESH: s2cloudless 'probability' band value > thresh = cloud
            
        :returns: img
        """
        # Get s2cloudless image, subset the probability band.
        cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

        # Condition s2cloudless by the probability threshold value.
        is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

        # Add the cloud probability layer and cloud mask as image bands.
        return img.addBands(ee.Image([cld_prb, is_cloud]))


    def add_shadow_bands(img):
        """ 
        identify cloud shadows from intersection of: 
            (1) darkest NIR scene pixels below NIR_DRK_THRESH that are not water
            (2) projected location of cloud shadows based on CLD_PRJ_DIST*10
        
        uses globals: 
            NIR_DRK_THRESH: if Band 8 (NIR) < NIR_DRK_THRESH = possible shadow
            CLD_PRJ_DIST:   max distnce [km or 100m?] from cloud edge for possible shadow  
            
        :returns: img
        """
        # Identify water pixels from the SCL band.
        not_water = img.select('SCL').neq(6)

        # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
        SR_BAND_SCALE = 1e4
        dark_pixels = (img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE)
                       .multiply(not_water)
                       .rename('dark_pixels')
                      )

        # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
        shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

        # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
        # mdj TODO: check why CLD_PRJ_DIST*10? i'm not convinced CLD_PRJ_DIST is in km.. 
        #           'clouds' is 10m res. 
        #           directionalDistanceTransform 2nd arg 'maxDistance' is in pixels
        #           so actually CLD_PRJ_DIST units = 100s of m?
        cld_proj = (img.select('clouds')
                    .directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
                    .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
            .select('distance')
            .mask()
            .rename('cloud_transform'))

        # Identify the intersection of dark pixels with cloud shadow projection.
        shadows = cld_proj.multiply(dark_pixels).rename('shadows')

        # Add dark pixels, cloud projection, and identified shadows as image bands.
        return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))
   

    def apply_cld_shdw_mask(img):
        """ 
        apply the cloud & shadow mask 
        :returns: img after application of cloud-shadow mask 
        """
        
        # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
        not_cld_shdw = img.select('cloudshadowmask').Not()

        # Subset reflectance bands and update their masks, return the result.
        return img.select('B.*').updateMask(not_cld_shdw)
    
    
    # get individual variables from param dict
    CLOUD_FILTER   = s2_params.get('CLOUD_FILTER')
    NIR_DRK_THRESH = s2_params.get('NIR_DRK_THRESH')
    CLD_PRJ_DIST   = s2_params.get('CLD_PRJ_DIST')
    CLD_PRB_THRESH = s2_params.get('CLD_PRB_THRESH')
    BUFFER         = s2_params.get('BUFFER')
    # mdj: S2BANDS is currently hard-coded here as not sure how to dynamically rename bands
    #S2BANDS        = s2_params.get('S2BANDS')
    S2BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
    
    # iteratively fetch each month of Sentinel-2 imagery and generate a median composite for the AOI
    for i, start_date in enumerate(start_date_list):
        #mnth=i+1
        print(f'fetch_sentinel2(): processing month {start_date}')
        end_date = ee.Date(start_date).advance(1, 'month')

        # load and filter collection
        s2_sr_cld_col = get_s2_sr_cld_col(aoi, start_date, end_date)

        # do cloud processing, make composite, clip and give sensible names
        s2cldless_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                     .map(apply_cld_shdw_mask)
                                     .select(S2BANDS) 
                                     .median()
                                     .clip(aoi.geometry())
                                     .rename(f'S2_B2_{start_date[0:7]}', 
                                             f'S2_B3_{start_date[0:7]}', 
                                             f'S2_B4_{start_date[0:7]}', 
                                             f'S2_B5_{start_date[0:7]}',
                                             f'S2_B6_{start_date[0:7]}', 
                                             f'S2_B7_{start_date[0:7]}', 
                                             f'S2_B8_{start_date[0:7]}', 
                                             f'S2_B8A_{start_date[0:7]}',
                                             f'S2_B11_{start_date[0:7]}', 
                                             f'S2_B12_{start_date[0:7]}'))    
        # append to stack
        if i == 0:
            median_stack = s2cldless_median
        else:
            median_stack = median_stack.addBands(s2cldless_median)
    
    print('fetch_sentinel2(): bye!')    
    return median_stack

In [6]:
def fetch_sentinel2_v2(aoi, date_list, s2_params):

    """
    fetch a datastack of Sentinel-2 composites, with cloud/shadow masking applied.
    most of the code to do this is derived from here:
    https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
    
    :: NEW FOR V2 ::
    * compositing period start and end dates need to be explicitly stated in 'date_list' (monthly composites no longer assumed).
    * bands returned are now defined by 's2_params', rather than hard coded.
    
    :param aoi: ee.featurecollection.FeatureCollection, used to indicate AOI extent 
    :param date_list: list of tuples of strings (i.e. [('a','b'),('c','d')]), used to define start & end of each compositing period, expects 'YYYY-MM-DD' format
    :param s2_params: dict, contains parameters used for cloud & shadow masking   
    :return: ee.image.Image, stack of monthly composite images of bands specified in s2_params
    """ 
    
    print('fetch_sentinel2(): hello!')
    
    def get_s2_sr_cld_col(aoi, start_date, end_date):
        """
        get & join the S2_SR and S2_CLOUD_PROBABILITY collections
        
        uses globals: 
            CLOUD_FILTER: max cloud coverage (%) permitted in a scene
        
        :returns: ee.ImageCollection
        """
        # Import and filter S2 SR.
        s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterBounds(aoi)
            .filterDate(start_date, end_date)
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

        # Import and filter s2cloudless.
        s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
            .filterBounds(aoi)
            .filterDate(start_date, end_date))

        # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
        return ee.ImageCollection(
            ee.Join.saveFirst('s2cloudless').apply(**{
                'primary': s2_sr_col,
                'secondary': s2_cloudless_col,
                'condition': ee.Filter.equals(**{
                    'leftField': 'system:index',
                    'rightField': 'system:index'
                })}))

    
    def add_cld_shdw_mask(img):
        """ 
        generate a cloud and shadow mask band 
        uses globals: 
            BUFFER: distance (m) used to buffer cloud edges
        :returns: img with added cloud mask, shadow mask, and cloud-shadow mask 
        """
        
        # Add cloud component bands.
        img_cloud = add_cloud_bands(img)

        # Add cloud shadow component bands.
        img_cloud_shadow = add_shadow_bands(img_cloud)

        # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
        is_cld_shdw = (img_cloud_shadow.select('clouds')
                       .add(img_cloud_shadow.select('shadows')).gt(0)
                      )

        # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
        # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
        # mdj TODO: confirmation that BUFFER is in [m]
        #           focal_max() default units = pixels (and pix res is 10m)
        #           so if BUFFER = 100
        #           100 * 0.1 = 10 pixels
        #           10 pix * 10 [pix res] = 100m
        is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
            .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
            .rename('cloudshadowmask'))

        # Add the final cloud-shadow mask to the image.
        return img_cloud_shadow.addBands(is_cld_shdw)    

       
    def add_cloud_bands(img):
        """
        identify cloudy pixels using s2cloudless product probabilty band
        
        uses globals:
            CLD_PRB_THRESH: s2cloudless 'probability' band value > thresh = cloud
            
        :returns: img
        """
        # Get s2cloudless image, subset the probability band.
        cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

        # Condition s2cloudless by the probability threshold value.
        is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

        # Add the cloud probability layer and cloud mask as image bands.
        return img.addBands(ee.Image([cld_prb, is_cloud]))


    def add_shadow_bands(img):
        """ 
        identify cloud shadows from intersection of: 
            (1) darkest NIR scene pixels below NIR_DRK_THRESH that are not water
            (2) projected location of cloud shadows based on CLD_PRJ_DIST*10
        
        uses globals: 
            NIR_DRK_THRESH: if Band 8 (NIR) < NIR_DRK_THRESH = possible shadow
            CLD_PRJ_DIST:   max distnce [km or 100m?] from cloud edge for possible shadow  
            
        :returns: img
        """
        # Identify water pixels from the SCL band.
        not_water = img.select('SCL').neq(6)

        # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
        SR_BAND_SCALE = 1e4
        dark_pixels = (img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE)
                       .multiply(not_water)
                       .rename('dark_pixels')
                      )

        # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
        shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

        # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
        # mdj TODO: check why CLD_PRJ_DIST*10? i'm not convinced CLD_PRJ_DIST is in km.. 
        #           'clouds' is 10m res. 
        #           directionalDistanceTransform 2nd arg 'maxDistance' is in pixels
        #           so actually CLD_PRJ_DIST units = 100s of m?
        cld_proj = (img.select('clouds')
                    .directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
                    .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
            .select('distance')
            .mask()
            .rename('cloud_transform'))

        # Identify the intersection of dark pixels with cloud shadow projection.
        shadows = cld_proj.multiply(dark_pixels).rename('shadows')

        # Add dark pixels, cloud projection, and identified shadows as image bands.
        return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))
   

    def apply_cld_shdw_mask(img):
        """ 
        apply the cloud & shadow mask 
        :returns: img after application of cloud-shadow mask 
        """
        
        # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
        not_cld_shdw = img.select('cloudshadowmask').Not()

        # Subset reflectance bands and update their masks, return the result.
        return img.select('B.*').updateMask(not_cld_shdw)
    
    
    
    # get individual variables from param dict
    CLOUD_FILTER   = s2_params.get('CLOUD_FILTER')
    NIR_DRK_THRESH = s2_params.get('NIR_DRK_THRESH')
    CLD_PRJ_DIST   = s2_params.get('CLD_PRJ_DIST')
    CLD_PRB_THRESH = s2_params.get('CLD_PRB_THRESH')
    BUFFER         = s2_params.get('BUFFER')
    S2BANDS        = s2_params.get('S2BANDS')
    
    # iteratively fetch each month of Sentinel-2 imagery and generate a median composite for the AOI
    for i, date_tuple in enumerate(date_list):
        print(f'fetch_sentinel2(): processing period: {date_tuple[0]} to {date_tuple[1]}')
        new_band_names = [f'S2_{x}_{date_tuple[0]}_{date_tuple[1]}' for x in S2BANDS]
        start_date = ee.Date(date_tuple[0])
        end_date = ee.Date(date_tuple[1])
        
        # load and filter collection
        s2_sr_cld_col = get_s2_sr_cld_col(aoi, start_date, end_date)

        # do cloud processing, make composite, clip and give sensible names
        s2cldless_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                     .map(apply_cld_shdw_mask)
                                     .select(S2BANDS) 
                                     .median()
                                     .clip(aoi.geometry())
                                     .rename(new_band_names))  
        
        # append to stack
        if i == 0:
            median_stack = s2cldless_median
        else:
            median_stack = median_stack.addBands(s2cldless_median)
    
    print('fetch_sentinel2(): bye!')    
    return median_stack

In [89]:
def fetch_sentinel2_v3(aoi, date_list, s2_params):

    """
    fetch a datastack of Sentinel-2 composites, with cloud/shadow masking applied.
    most of the code to do this is derived from here:
    https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
    
    :: NEW FOR V2 ::
    * compositing period start and end dates need to be explicitly stated in 'date_list' (monthly composites no longer assumed).
    * bands returned are now defined by 's2_params', rather than hard coded.

    :: NEW FOR V3 ::
    * attempts to fill cloud gaps with same time window of data from the previous year 
      NOTE: this is not applied prior to April 2018 (no sentinel data on GEE prior to Apr 2017)

    :param aoi: ee.featurecollection.FeatureCollection, used to indicate AOI extent 
    :param date_list: list of tuples of strings (i.e. [('a','b'),('c','d')]), used to define start & end of each compositing period, expects 'YYYY-MM-DD' format
    :param s2_params: dict, contains parameters used for cloud & shadow masking   
    :return: ee.image.Image, stack of monthly composite images of bands specified in s2_params
    """ 
    
    print('fetch_sentinel2(): hello!')
    
    def get_s2_sr_cld_col(aoi, start_date, end_date):
        """
        get & join the S2_SR and S2_CLOUD_PROBABILITY collections
        
        uses globals: 
            CLOUD_FILTER: max cloud coverage (%) permitted in a scene
        
        :returns: ee.ImageCollection
        """
        # Import and filter S2 SR.
        s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterBounds(aoi)
            .filterDate(start_date, end_date)
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

        # Import and filter s2cloudless.
        s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
            .filterBounds(aoi)
            .filterDate(start_date, end_date))

        # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
        return ee.ImageCollection(
            ee.Join.saveFirst('s2cloudless').apply(**{
                'primary': s2_sr_col,
                'secondary': s2_cloudless_col,
                'condition': ee.Filter.equals(**{
                    'leftField': 'system:index',
                    'rightField': 'system:index'
                })}))

    
    def add_cld_shdw_mask(img):
        """ 
        generate a cloud and shadow mask band 
        uses globals: 
            BUFFER: distance (m) used to buffer cloud edges
        :returns: img with added cloud mask, shadow mask, and cloud-shadow mask 
        """
        
        # Add cloud component bands.
        img_cloud = add_cloud_bands(img)

        # Add cloud shadow component bands.
        img_cloud_shadow = add_shadow_bands(img_cloud)

        # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
        is_cld_shdw = (img_cloud_shadow.select('clouds')
                       .add(img_cloud_shadow.select('shadows')).gt(0)
                      )

        # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
        # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
        # mdj TODO: confirmation that BUFFER is in [m]
        #           focal_max() default units = pixels (and pix res is 10m)
        #           so if BUFFER = 100
        #           100 * 0.1 = 10 pixels
        #           10 pix * 10 [pix res] = 100m
        is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
            .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
            .rename('cloudshadowmask'))

        # Add the final cloud-shadow mask to the image.
        return img_cloud_shadow.addBands(is_cld_shdw)    

       
    def add_cloud_bands(img):
        """
        identify cloudy pixels using s2cloudless product probabilty band
        
        uses globals:
            CLD_PRB_THRESH: s2cloudless 'probability' band value > thresh = cloud
            
        :returns: img
        """
        # Get s2cloudless image, subset the probability band.
        cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

        # Condition s2cloudless by the probability threshold value.
        is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

        # Add the cloud probability layer and cloud mask as image bands.
        return img.addBands(ee.Image([cld_prb, is_cloud]))


    def add_shadow_bands(img):
        """ 
        identify cloud shadows from intersection of: 
            (1) darkest NIR scene pixels below NIR_DRK_THRESH that are not water
            (2) projected location of cloud shadows based on CLD_PRJ_DIST*10
        
        uses globals: 
            NIR_DRK_THRESH: if Band 8 (NIR) < NIR_DRK_THRESH = possible shadow
            CLD_PRJ_DIST:   max distnce [km or 100m?] from cloud edge for possible shadow  
            
        :returns: img
        """
        # Identify water pixels from the SCL band.
        not_water = img.select('SCL').neq(6)

        # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
        SR_BAND_SCALE = 1e4
        dark_pixels = (img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE)
                       .multiply(not_water)
                       .rename('dark_pixels')
                      )

        # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
        shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

        # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
        # mdj TODO: check why CLD_PRJ_DIST*10? i'm not convinced CLD_PRJ_DIST is in km.. 
        #           'clouds' is 10m res. 
        #           directionalDistanceTransform 2nd arg 'maxDistance' is in pixels
        #           so actually CLD_PRJ_DIST units = 100s of m?
        cld_proj = (img.select('clouds')
                    .directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
                    .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
            .select('distance')
            .mask()
            .rename('cloud_transform'))

        # Identify the intersection of dark pixels with cloud shadow projection.
        shadows = cld_proj.multiply(dark_pixels).rename('shadows')

        # Add dark pixels, cloud projection, and identified shadows as image bands.
        return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))
   

    def apply_cld_shdw_mask(img):
        """ 
        apply the cloud & shadow mask 
        :returns: img after application of cloud-shadow mask 
        """
        
        # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
        not_cld_shdw = img.select('cloudshadowmask').Not()

        # Subset reflectance bands and update their masks, return the result.
        return img.select('B.*').updateMask(not_cld_shdw)
    
    
    
    # get individual variables from param dict
    CLOUD_FILTER   = s2_params.get('CLOUD_FILTER')
    NIR_DRK_THRESH = s2_params.get('NIR_DRK_THRESH')
    CLD_PRJ_DIST   = s2_params.get('CLD_PRJ_DIST')
    CLD_PRB_THRESH = s2_params.get('CLD_PRB_THRESH')
    BUFFER         = s2_params.get('BUFFER')
    S2BANDS        = s2_params.get('S2BANDS')
    
    # iteratively fetch each month of Sentinel-2 imagery and generate a median composite for the AOI
    for i, date_tuple in enumerate(date_list):
        print(f'fetch_sentinel2(): processing period: {date_tuple[0]} to {date_tuple[1]}')
        new_band_names = [f'S2_{x}_{date_tuple[0]}_{date_tuple[1]}' for x in S2BANDS]
        start_date = ee.Date(date_tuple[0])
        end_date = ee.Date(date_tuple[1])
        
        # load and filter collection
        s2_sr_cld_col = get_s2_sr_cld_col(aoi, start_date, end_date)
        # do cloud processing, make composite & clip.
        s2cldless_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                     .map(apply_cld_shdw_mask)
                                     .select(S2BANDS) 
                                     .median()
                                     .clip(aoi.geometry()))       
        
        # try to cloud gap fill 
        if dt.datetime.strptime(date_tuple[0], '%Y-%m-%d') > dt.datetime.strptime('2018-03-28', '%Y-%m-%d'):
            # load a collection from the same time in previous year for cloud gap filling
            s2_sr_cld_col_fill = get_s2_sr_cld_col(aoi, start_date.advance(-1, 'year'), end_date.advance(-1, 'year'))
            # do cloud processing, make composite & clip.
            s2cldless_median_fill = (s2_sr_cld_col_fill.map(add_cld_shdw_mask)
                                         .map(apply_cld_shdw_mask)
                                         .select(S2BANDS) 
                                         .median()
                                         .clip(aoi.geometry()))
            # apply cloud gap filling                    
            s2cldless_median = fill_cloud_gaps(img_orig=s2cldless_median, 
                                               img_fill=s2cldless_median_fill)
        else:
            print(f"fetch_sentinel2():Skipping cloud gap filling; no S2 data prior to 2017-03-28 available in GEE, cannot fill cloud gaps for {date_tuple[0]}-{date_tuple[1]} with previous year of data")
                   
        # rename bands
        s2cldless_median = s2cldless_median.rename(new_band_names)  
        
        # append to stack
        if i == 0:
            median_stack = s2cldless_median
        else:
            median_stack = median_stack.addBands(s2cldless_median)
    
    print('fetch_sentinel2(): bye!')    
    return median_stack

In [80]:
import datetime as dt

dt.datetime.strptime(date_tuple[0], '%Y-%m-%d') < dt.datetime.strptime('2018-04-01', '%Y-%m-%d')



('2018-01-01', '2018-01-31')

In [7]:
def map_sentinel1(stack, start_date_list, lat=51.85, lon=27.8):
    """
    Quick mapping function for debugging S1 data (NOTE: VH duplicated in Green & Blue)
    : param stack: Image, stack of S1 composite images for AOI
    : param start_date_list: list, strings used to define start of each month, expects 'YYYY-MM-01' format
    : param lat: float, map central latitude
    : param lon: float, map central longitude
    : return : 
    """
    Map = geemap.Map(center=(lat, lon), zoom=9)
    Map.add_basemap('SATELLITE')
    for i, start_date in enumerate(start_date_list):
        vis = {'min': -50,'max': 1, 'bands': [f'S1_VV_{start_date[0:7]}', 
                                              f'S1_VH_{start_date[0:7]}', 
                                              f'S1_VH_{start_date[0:7]}']}
        Map.addLayer(stack, vis, f'S1_{start_date}')
    return Map

In [8]:
def map_sentinel2(stack, start_date_list, lat=51.85, lon=27.8):
    """
    Quick mapping function for debugging S2 data - RGB only
    : param stack: Image, stack of S2 composite images for AOI
    : param start_date_list: list, strings used to define start of each month, expects 'YYYY-MM-01' format
    : param lat: float, map central latitude
    : param lon: float, map central longitude
    : return : 
    """
    Map = geemap.Map(center=(lat, lon), zoom=9)
    Map.add_basemap('SATELLITE')
    for i, start_date in enumerate(start_date_list):
        vis = {'min': -0.0,'max': 3000, 'bands': [f'S2_B4_{start_date[0:7]}', 
                                                  f'S2_B3_{start_date[0:7]}', 
                                                  f'S2_B2_{start_date[0:7]}']}
        Map.addLayer(stack, vis, f'S2_{start_date}')
    return Map

In [9]:
def create_data_stack(aoi, start_date_list, s2_params):
    """ 
    convience function to compile and combine all distinct dataset sub-stacks
    * Sentinel 1 data bands: 'VV', 'VH' [monthly]
    * Sentinel 2 data bands:'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12' [monthly]

    :param aoi: ee.featurecollection.FeatureCollection, used to indicate AOI extent 
    :param start_date_list: list, strings used to define start of each month, expects 'YYYY-MM-01' format
    :param s2_params: dict, contains parameters used for cloud & shadow masking   
    :return: ee.image.Image

    """
    s1_stack = fetch_sentinel1_v1(aoi, start_date_list)
    s2_stack = fetch_sentinel2_v1(aoi, start_date_list, s2_params)
    combined_stack = s1_stack.addBands(s2_stack)
    return combined_stack


In [48]:
def fill_cloud_gaps(img_orig, img_fill):
    """
    Where img_orig is masked (i.e. transparent null values) due to e.g. cloud masking, 
    fill those gaps where possible using data from img_fill. any remaining gaps (i.e. cloudy in both images)
    are re-masked.
    
    :param img_orig: Image, to be filled 
    :param img_fill: Image, used for filling
    :returns: img_new, img_orig after gap filling and remasking
    """
    img_new = img_orig.unmask(-99999) # masked locations
    fill_pixels = img_new.eq(-99999)  # binary mask with value = 1 where we want to fill
    img_new = img_new.where(fill_pixels, img_fill) # fill img_new with img_fill where fill_pixels==1
    mask = img_new.neq(-99999) # -99999 will remain where no valid pixels in img_fill (i.e. cloudy in both), so remask
    img_new = img_new.mask(mask)
    return img_new

In [10]:
# variables - could all easily be added to a config file somewhere

fp_train_ext = "/home/markdj/Dropbox/artio/polesia/val/Vegetation_extent_rough.shp"
aoi = geemap.shp_to_ee(fp_train_ext)
start_date_list = ['2019-01-01', '2019-02-01', 
                   '2019-03-01', '2019-04-01', 
                   '2019-05-01', '2019-06-01', 
                   '2019-07-01', '2019-08-01',
                   '2019-09-01', '2019-10-01',
                   '2019-11-01', '2019-12-01']

s2_params = {
    'CLOUD_FILTER': 60,       # int, max cloud coverage (%) permitted in a scene
    'CLD_PRB_THRESH': 40,     # int, 's2cloudless' 'probability' band value > thresh = cloud
    'NIR_DRK_THRESH': 0.15,   # float, if Band 8 (NIR) < NIR_DRK_THRESH = possible shadow
    'CLD_PRJ_DIST': 1,        # int, max distnce [TODO: km or 100m?] from cloud edge for possible shadow 
    'BUFFER': 100,            # int, distance (m) used to buffer cloud edges
    #'S2BANDS': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'] #list of str, which S2 bands to return?
    'S2BANDS': ['B2', 'B3', 'B4'] #list of str, which S2 bands to return?

}

In [57]:
# get S1
s1_stack = fetch_sentinel1_v1(aoi, start_date_list)
s1map = map_sentinel1(s1_stack, start_date_list)
s1map

fetch_sentinel1(): hello!
fetch_sentinel1(): processing month 2019-01-01
fetch_sentinel1(): processing month 2019-02-01
fetch_sentinel1(): processing month 2019-03-01
fetch_sentinel1(): processing month 2019-04-01
fetch_sentinel1(): processing month 2019-05-01
fetch_sentinel1(): processing month 2019-06-01
fetch_sentinel1(): processing month 2019-07-01
fetch_sentinel1(): processing month 2019-08-01
fetch_sentinel1(): processing month 2019-09-01
fetch_sentinel1(): processing month 2019-10-01
fetch_sentinel1(): processing month 2019-11-01
fetch_sentinel1(): processing month 2019-12-01
fetch_sentinel1(): bye!


Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [11]:
# get S2
s2_stack = fetch_sentinel2_v1(aoi, start_date_list, s2_params)
s2map = map_sentinel2(s2_stack, start_date_list)
s2map

fetch_sentinel2(): hello!
fetch_sentinel2(): processing month 2019-01-01
fetch_sentinel2(): processing month 2019-02-01
fetch_sentinel2(): processing month 2019-03-01
fetch_sentinel2(): processing month 2019-04-01
fetch_sentinel2(): processing month 2019-05-01
fetch_sentinel2(): processing month 2019-06-01
fetch_sentinel2(): processing month 2019-07-01
fetch_sentinel2(): processing month 2019-08-01
fetch_sentinel2(): processing month 2019-09-01
fetch_sentinel2(): processing month 2019-10-01
fetch_sentinel2(): processing month 2019-11-01
fetch_sentinel2(): processing month 2019-12-01
fetch_sentinel2(): bye!


Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

### testing the v2 versions of fetch s1&2

In [11]:
#help(fetch_sentinel1_v2)

date_list = [('2018-12-01', '2019-02-01'),('2019-05-01', '2019-05-31')]
date_list = [('2018-12-01', '2019-02-01')]

s2tmp = fetch_sentinel2_v2(aoi, date_list, s2_params)
#s2tmp.getInfo()
s1tmp = fetch_sentinel1_v2(aoi, date_list)
#s1tmp.getInfo()

# but of course, now i have changed band naming so the mapping functions are screwed again...
#calling get info on everything is very useful!
s1bands = s1tmp.bandNames().getInfo()
s2bands = s2tmp.bandNames().getInfo()
print(s2bands)

lat=51.85
lon=27.8

Map = geemap.Map(center=(lat, lon), zoom=9)
Map.add_basemap('SATELLITE')

vis = {'min': -50,'max': 1, 'bands': [s1bands[0],s1bands[1],s1bands[1]]}
Map.addLayer(s1tmp, vis, f'S1')
vis = {'min': -1,'max': 3000, 'bands': [s2bands[2],s2bands[1],s2bands[0]]}
Map.addLayer(s2tmp, vis, f'S2')

Map

fetch_sentinel2(): hello!
fetch_sentinel2(): processing period: 2018-12-01 to 2019-02-01
fetch_sentinel2(): bye!
fetch_sentinel1(): hello!
fetch_sentinel1(): processing period: 2018-12-01 to 2019-02-01
fetch_sentinel1(): bye!
['S2_B2_2018-12-01_2019-02-01', 'S2_B3_2018-12-01_2019-02-01', 'S2_B4_2018-12-01_2019-02-01']


Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

## cloud gap filling
* how do we deal with NaN areas due to cloud? try to fill with data for same period in different year
* here suggests 'where' can be used to fill like this: https://gis.stackexchange.com/questions/323728/smoothing-interpolating-across-images-in-an-imagecollection-to-remove-missing-da
* also see here https://code.earthengine.google.com/17ee7142a98fdb1c37b7da4aa679587c


not working yet

In [13]:

start_date_list = ['2019-01-01']
s2_stack19 = fetch_sentinel2_v1(aoi, start_date_list, s2_params)
start_date_list = ['2018-01-01']
s2_stack18 = fetch_sentinel2_v1(aoi, start_date_list, s2_params)

#this does not work ======
new=s2_stack18.where(s2_stack19, s2_stack19)

lat=51.85
lon=27.8

Map = geemap.Map(center=(lat, lon), zoom=9)
Map.add_basemap('SATELLITE')
vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2019-01', f'S2_B3_2019-01', f'S2_B2_2019-01']}
Map.addLayer(s2_stack19, vis, f'S2_rgb_2019-04')
vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2018-01', f'S2_B3_2018-01', f'S2_B2_2018-01']}
Map.addLayer(s2_stack18, vis, f'S2_rgb_2018-04')
vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2018-01', f'S2_B3_2018-01', f'S2_B2_2018-01']}
Map.addLayer(new, vis, f'2018filled')
Map



fetch_sentinel2(): hello!
fetch_sentinel2(): processing month 2019-01-01
fetch_sentinel2(): bye!
fetch_sentinel2(): hello!
fetch_sentinel2(): processing month 2018-01-01
fetch_sentinel2(): bye!


Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [25]:
fp_train_ext = "/home/markdj/Dropbox/artio/polesia/val/tiny_roi.shp"
aoi = geemap.shp_to_ee(fp_train_ext)

start_date_list = ['2019-01-01']
s2_stack19 = fetch_sentinel2_v1(aoi, start_date_list, s2_params)
start_date_list = ['2018-01-01']
s2_stack18 = fetch_sentinel2_v1(aoi, start_date_list, s2_params)

fetch_sentinel2(): hello!
fetch_sentinel2(): processing month 2019-01-01
fetch_sentinel2(): bye!
fetch_sentinel2(): hello!
fetch_sentinel2(): processing month 2018-01-01
fetch_sentinel2(): bye!


In [28]:
lat=51.85
lon=27.8

Map = geemap.Map(center=(lat, lon), zoom=9)
#Map.add_basemap('SATELLITE')
vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2019-01', f'S2_B3_2019-01', f'S2_B2_2019-01']}
Map.addLayer(s2_stack19, vis, f's2_stack19')

vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2018-01', f'S2_B3_2018-01', f'S2_B2_2018-01']}
Map.addLayer(s2_stack18, vis, f's2_stack18')

Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [ ]:
#new=s2_stack18nulls.where(invalid, s2_stack19)
    # new is s2_stack18 filled with -9999


In [29]:
#s2_stack18nulls.getInfo()
#.bandNames().getInfo()
#invalid.getInfo()

**fill s2_stack18nulls with s2_stack19.**

* in places where 'invalid18'==1 and s2_stack19 is 'null' (ie cloud in both), behaviour seems to use -9999 value from somewhere

* but where? 'new2' generation involves no layers with -9999 in it but has them nontheless?

In [47]:
# Nulls = s2_stack19.unmask(-9999)
# Map = geemap.Map(center=(lat, lon), zoom=9)
# Map.add_basemap('SATELLITE')
# vis = {'min': -999,'max': 3000,'bands': [f'S2_B4_2019-01', f'S2_B3_2019-01', f'S2_B2_2019-01']}
# Map.addLayer(Nulls, vis, f'2018filled')
# Map

# set the null values to -9999
s2_stack18nulls = s2_stack18.unmask(-99999)
s2_stack19nulls = s2_stack19.unmask(-99999)

# generates a binary mask where null (-9999) areas become 1
#invalid = s2_stack19nulls.lt(-9000)
invalid18 = s2_stack18nulls.eq(-99999)

new = s2_stack18nulls.where(invalid18, s2_stack19)
#new2 = s2_stack18.where(invalid18, s2_stack19) # does nothing. presumably cant replace 'null' value using where, only a fill(-9999) val?
mask = new.neq(-99999) #make a new mask for the still unfilled areas
new3 = new.mask(mask)


lat=51.85
lon=27.8
Map = geemap.Map(center=(lat, lon), zoom=9)
#Map.add_basemap('SATELLITE')

# vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2019-01', f'S2_B3_2019-01', f'S2_B2_2019-01']}
# Map.addLayer(s2_stack19, vis, f's2_stack19')
# vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2018-01', f'S2_B3_2018-01', f'S2_B2_2018-01']}
# Map.addLayer(s2_stack18, vis, f's2_stack18')

vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2018-01', f'S2_B3_2018-01', f'S2_B2_2018-01']}
Map.addLayer(s2_stack18nulls, vis, f's2_stack18nulls')

vis = {'min': 0,'max': 1,'bands': [f'S2_B4_2018-01']}
Map.addLayer(invalid18, vis, f'invalid18')

vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2018-01', f'S2_B3_2018-01', f'S2_B2_2018-01']}
Map.addLayer(new, vis, f'new18')
#Map.addLayer(new2, vis, f'new18_v2')

vis = {'min': 0,'max': 1,'bands': [f'S2_B4_2018-01']}
Map.addLayer(mask, vis, f'mask')

vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2018-01', f'S2_B3_2018-01', f'S2_B2_2018-01']}
Map.addLayer(new3, vis, f'new18_v3')


img_new = fill_cloud_gaps(img_orig=s2_stack18, img_fill=s2_stack19)


vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2018-01', f'S2_B3_2018-01', f'S2_B2_2018-01']}
Map.addLayer(img_new, vis, f'img_new_v3')
Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [90]:
fp_train_ext = "/home/markdj/Dropbox/artio/polesia/val/tiny_roi.shp"
aoi = geemap.shp_to_ee(fp_train_ext)

start_date_list = [('2018-01-01','2018-01-31')]
s2_stack18 = fetch_sentinel2_v2(aoi, start_date_list, s2_params)
s2_stack18fill = fetch_sentinel2_v3(aoi, start_date_list, s2_params)

start_date_list = [('2019-01-01','2019-01-31')]
s2_stack19 = fetch_sentinel2_v2(aoi, start_date_list, s2_params)
s2_stack19fill = fetch_sentinel2_v3(aoi, start_date_list, s2_params)

fetch_sentinel2(): hello!
fetch_sentinel2(): processing period: 2018-01-01 to 2018-01-31
fetch_sentinel2(): bye!
fetch_sentinel2(): hello!
fetch_sentinel2(): processing period: 2018-01-01 to 2018-01-31
fetch_sentinel2():Skipping cloud gap filling; no S2 data prior to 2017-03-28 available in GEE, cannot fill cloud gaps for 2018-01-01-2018-01-31 with previous year of data
fetch_sentinel2(): bye!
fetch_sentinel2(): hello!
fetch_sentinel2(): processing period: 2019-01-01 to 2019-01-31
fetch_sentinel2(): bye!
fetch_sentinel2(): hello!
fetch_sentinel2(): processing period: 2019-01-01 to 2019-01-31
fetch_sentinel2(): bye!


In [60]:
# date_tuple=('2018-01-01','2018-01-31')
# start_date = ee.Date(date_tuple[0])
# end_date = ee.Date(date_tuple[1])
# # load a collection from the same time in previous year for cloud gap filling
# print(start_date.advance(-1, 'year').format().getInfo(), end_date.advance(-1, 'year').format().getInfo())


2017-01-01T00:00:00 2017-01-31T00:00:00


In [91]:
lat=51.85
lon=27.8

Map = geemap.Map(center=(lat, lon), zoom=9)
#Map.add_basemap('SATELLITE')
vis = {'min': -0.0,'max': 3000,'bands': ['S2_B2_2018-01-01_2018-01-31', 'S2_B3_2018-01-01_2018-01-31', 'S2_B4_2018-01-01_2018-01-31']}
Map.addLayer(s2_stack18, vis, f's2_stack18')

vis = {'min': -0.0,'max': 3000,'bands': ['S2_B2_2019-01-01_2019-01-31', 'S2_B3_2019-01-01_2019-01-31', 'S2_B4_2019-01-01_2019-01-31']}
Map.addLayer(s2_stack19, vis, f's2_stack19')
Map.addLayer(s2_stack19fill, vis, f's2_stack19fill')

Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

other stuff

In [12]:
#sss = s2_stack.addBands(s1_stack)
#sss.getInfo()

stack = create_data_stack(aoi, start_date_list, s2_params)
stack.getInfo()

lat=51.85
lon=27.8

Map = geemap.Map(center=(lat, lon), zoom=9)
Map.add_basemap('SATELLITE')

vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B4_2019-04', f'S2_B3_2019-04', f'S2_B2_2019-04']}
Map.addLayer(stack, vis, f'S2_rgb_2019-04')
vis = {'min': -0.0,'max': 3000,'bands': [f'S2_B8_2019-04', f'S2_B4_2019-04', f'S2_B3_2019-04']}
Map.addLayer(stack, vis, f'S2_fcc_2019-04')
vis = {'min': -0.0,'max': 3000, 'bands': [f'S2_B8_2019-04']}
Map.addLayer(stack, vis, f'S2_B8_2019-04')
vis = {'min': -0.0,'max': 3000, 'bands': [f'S2_B12_2019-04']}
Map.addLayer(stack, vis, f'S2_B12_2019-04')
vis = {'min': -50,'max': 1, 'bands': [f'S1_VV_2019-04', f'S1_VH_2019-04', f'S1_VH_2019-04']}
Map.addLayer(stack, vis, f'S1_VV_VH_2019-04')
Map

fetch_sentinel1(): hello!
fetch_sentinel1(): processing month 2019-01-01
fetch_sentinel1(): processing month 2019-02-01
fetch_sentinel1(): processing month 2019-03-01
fetch_sentinel1(): processing month 2019-04-01
fetch_sentinel1(): processing month 2019-05-01
fetch_sentinel1(): processing month 2019-06-01
fetch_sentinel1(): processing month 2019-07-01
fetch_sentinel1(): processing month 2019-08-01
fetch_sentinel1(): processing month 2019-09-01
fetch_sentinel1(): processing month 2019-10-01
fetch_sentinel1(): processing month 2019-11-01
fetch_sentinel1(): processing month 2019-12-01
fetch_sentinel1(): bye!
fetch_sentinel2(): hello!
fetch_sentinel2(): processing month 2019-01-01
fetch_sentinel2(): processing month 2019-02-01
fetch_sentinel2(): processing month 2019-03-01
fetch_sentinel2(): processing month 2019-04-01
fetch_sentinel2(): processing month 2019-05-01
fetch_sentinel2(): processing month 2019-06-01
fetch_sentinel2(): processing month 2019-07-01
fetch_sentinel2(): processing m

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…